### Types of Joins
- `Inner Join` - Returns rows that match on both table A and B for example if you have *customers* and *orders* tables it will only return the data on `customers` with at least **one** order placed.
- `Left Join` - Returns all the rows from Table A and matching data from Table B, ie even if a customer has never placed an order they should still appear in the output if Tabele A was *customers* table.
- `Right Join`- Returns all the rows from Table B and matching data from Table A, ie even if a customer has never placed an order they should still appear in the output if Table B was *customers* table and *orders* is table A.
- `Full Outer Join` - Returns all rows regardless of matching and fill in null values for non-matching.
- `Cross Join` - mixing everything with everything if you have 5 rows in A and 6 rows in B you will get 5*6 = 30 rows.

In [ ]:
# what happens if there is a matching row
# in the customers table to multiple orders in the orders table
#do we expect all 100 rows from customers (A) and 120 rows from orders(B)

""" 
220
130
150
"""

In [1]:
import sqlite3
import pandas as pd


#connection
conn = sqlite3.connect('./data/data.sqlite')

In [ ]:
### left join

# customerName, phone
# orderDate, orderNumber

query1 = """ 
SELECT c.customerName, c.phone,
       o.orderDate, o.orderNumber
FROM customers c
LEFT JOIN orders o
USING(customerNumber);
"""

customer_orders_df = pd.read_sql_query(query1, conn)

customer_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customerName  350 non-null    object 
 1   phone         350 non-null    object 
 2   orderDate     326 non-null    object 
 3   orderNumber   326 non-null    float64
dtypes: float64(1), object(3)
memory usage: 11.1+ KB


In [3]:
customer_orders_df.head()

,customerName,phone,orderDate,orderNumber
0,Atelier graphique,40.32.2555,2003-05-20,10123.0
1,Atelier graphique,40.32.2555,2004-09-27,10298.0
2,Atelier graphique,40.32.2555,2004-11-25,10345.0
3,Signal Gift Stores,7025551838,2003-05-21,10124.0
4,Signal Gift Stores,7025551838,2004-08-06,10278.0


The left join gets all data from the customers' table even if they dont have an order yet.

In [12]:
#left join with aggregate function
#Count orders made by customers 
query2 = """ 
SELECT c.customerName,
       COUNT(o.orderNumber) AS orders
FROM customers c
LEFT JOIN orders o
USING(customerNumber)
GROUP BY c.customerNumber
ORDER BY orders DESC;
"""


amnt_orders_df = pd.read_sql_query(query2, conn)

amnt_orders_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customerName  122 non-null    object
 1   orders        122 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


In [13]:
amnt_orders_df.tail(10)

,customerName,orders
112,Schuyler Imports,0
113,BG&E Collectables,0
114,"Franken Gifts, Co",0
115,Messner Shopping Network,0
116,ANG Resellers,0
117,Natürlich Autos,0
118,"Asian Shopping Network, Co",0
119,Porto Imports Co.,0
120,American Souvenirs Inc,0
121,Havel & Zbyszek Co,0


In [29]:
conn2.close()

In [31]:
""" 
Tables:
* students
* courses
* projects
"""


#connection & cursor
conn2 = sqlite3.connect('./dspt13.sqlite')


#cursor
cursor = conn2.cursor()


In [16]:
""" 
Tables:
* students
* courses
* projects
"""


Q1 = """ 
CREATE TABLE students (
        student_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL, 
        age INTEGER
);
"""
cursor.execute(Q1)




In [28]:
pd.read_sql_query("""
SELECT name 
FROM sqlite_master
WHERE type = 'table';
""", conn2)

,name
0,students
1,courses
2,projects


In [18]:
pd.read_sql_query("""
SELECT *
FROM students;
""", conn2)

,student_id,name,age


In [19]:
""" 
Tables:
* students
* courses
* projects
"""

Q2 = """ 
CREATE TABLE courses (
        course_id INTEGER PRIMARY KEY,
        title TEXT NOT NULL, 
        duration INTEGER
);
"""
cursor.execute(Q2)

In [20]:
pd.read_sql_query("""
SELECT name 
FROM sqlite_master
WHERE type = 'table';
""", conn2)

,name
0,students
1,courses


In [32]:
""" 
Tables:
* students
* courses
* projects
"""
#one student many projects(one to many relationship)
Q3 = """ 
CREATE TABLE IF NOT EXISTS projects (
        project_id INTEGER PRIMARY KEY, 
        title TEXT,
        studentID INTEGER,
    FOREIGN KEY(studentID) REFERENCES students(student_id)
);
"""
cursor.execute(Q3)

In [34]:
pd.read_sql_query("""
SELECT name 
FROM sqlite_master
WHERE type = 'table';
""", conn2)

,name
0,students
1,courses
2,projects
3,students_courses


In [33]:
""" 
Tables:
* students
* courses
* projects
"""
#one student many courses & one course many students(many to many relationship)
#junction table
Q4 = """ 
CREATE TABLE IF NOT EXISTS students_courses(
        student_id INTEGER,
        course_id INTEGER,
        PRIMARY KEY(student_id, course_id),
    FOREIGN KEY(student_id) REFERENCES students(student_id),
    FOREIGN KEY(course_id) REFERENCES courses(course_id)
);
"""
cursor.execute(Q4)

In [35]:
conn2.commit() #submits changes to the file itself

conn2.close()